In [27]:
import os

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [28]:
from crewai import Agent
from crewai_tools import FileReadTool, DirectoryReadTool


class Agents:
    def pathSearcher(self):
        return Agent(
            role="pathSearcher",
            goal="Finds the markdown files inside {file_path} path",
            backstory="You are fluent in Korean, and you are very good at finding files.",
            verbose=True,
            tools=[
                DirectoryReadTool(),
            ],
        )

    def researcher(self):
        return Agent(
            role="Researcher",
            goal="Finds and summarizes the markdown files that can solve the {question}",
            backstory="You are fluent in Korean, and you are very good at finding and summarizing files. Summarized information is essential for problem solving",
            verbose=True,
            tools=[
                FileReadTool(),
            ],
            max_iter=10,
        )

In [29]:
from crewai import Task


class Tasks:
    def pathSearch(self, agent):
        return Task(
            description="Finds the markdown files inside {file_path} path",
            expected_output="Your final answer MUST relative path",
            agent=agent,
            output_file="algorithMarkdownPath.md",
        )

    def research(self, agent, context):
        return Task(
            description="Based on the pathSearch, Gather and analyze the most relevant markdown files for troubleshooting {question}. Provide a summary of markdown files and what is an important point in files. If there is a symbol '[[file_name]]' in the markdown file, you should also look into the markdown file with the file_name.",
            expected_output="Your final answer MUST be a detailed summary of the markdown files in Korean. Include Path to all imported files, why the file was imported, what is an important point and If there is an example for {question}, include it. If you can't find the file, JUST Answer that you don't know",
            agent=agent,
            context=context,
            output_file="questionAnswer.md",
        )

In [32]:
from crewai import Crew

agent = Agents()
tasks = Tasks()

pathSearcher = agent.pathSearcher()
researcher = agent.researcher()

pathSearcher_task = tasks.pathSearch(pathSearcher)
researcher_task = tasks.research(researcher, [pathSearcher_task])

crew = Crew(
    agents=[
        pathSearcher,
        researcher,
    ],
    tasks=[
        pathSearcher_task,
        researcher_task,
    ],
    verbose=2,
    memory=True,
)

result = crew.kickoff(
    dict(
        file_path="./Algorithm/Algorithm Content",
        question="MST 예시 코드를 보여줘",
    )
)

2025-01-06 02:15:30,176 - 21476 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: pathSearcher
 [INFO]: == Starting Task: Finds the markdown files inside ./Algorithm/Algorithm Content path


> Entering new CrewAgentExecutor chain...
I need to find markdown files located in the './Algorithm/Algorithm Content' directory. To begin, I will list the files in that directory to see what I have available.

Action: List files in directory  
Action Input: {"directory": "./Algorithm/Algorithm Content"}   

File paths: 
-./Algorithm/Algorithm Content/Array\Binary Search.md
- ./Algorithm/Algorithm Content/Array\MITM(Meet in the middle).md
- ./Algorithm/Algorithm Content/Array\PBS(Parallel Binary Search).md
- ./Algorithm/Algorithm Content/Graph Theory\Articulation Points And Bridges.md
- ./Algorithm/Algorithm Content/Graph Theory\BFS(Breadth-First Search).md
- ./Algorithm/Algorithm Content/Graph Theory\CCW(Counter Clock Wise).md
- ./Algorithm/Algorithm Content/Graph Theory\DFS(Depth-First Search).md
- ./Algorithm/Algorithm Content/Graph Theory\Dijkstra